# Simulate a Kalman filter for fixed position receivers

This needs a 5 state model

In [1]:
import numpy as np
import scipy.constants as const

In [18]:
# first generate a scenario generator

# A bunch of satellites/beacons
# have a device with unknown time and frequency
# generate a series of readings

# use satellites positions from old example

satellitePositions = np.array([
        [22808160.9, -12005866.6, -6609526.5],
        [21141179.5, -2355056.3, -15985716.1],
        [20438959.3, -4238967.1, 16502090.2],
        [18432296.2, -18613382.5, -4672400.8],
        [21772117.8, 13773269.7, 6656636.4],
        [15561523.9, 3469098.6, -21303596.2],
        [13773316.6, 15929331.4, -16266254.4]
    ])

def generateMeasurement(time, truePos, clockParams, almanac):
    distance = np.linalg.norm(almanac - truePos, axis = 1)
    measurement = distance[:,None] + time[None,:]*clockParams[1]
    return measurement

def noisyMeasurement(time, truePos, clockParams, almanac, sigma):
    measurement = generateMeasurement(time, truePos, clockParams, almanac)
    noise = np.random.normal(0, sigma, size = (almanac.shape[0], time.shape[0]))
    return measurement + noise

# ground truth: we don't know this but would want to estimate it
xTrue = np.array([6378137.0, 0.0, 0.0])
startClockBias = 85000.0
clockFreqOffset = 4.5e-6 # a +4.5 ppm clock offset
clockFreq = const.c*(1+clockFreqOffset)
measurementNoiseSigma = 6
startingClockParams = np.array([startClockBias, clockFreq]) 

np.random.seed(42)

measurements = noisyMeasurement(np.arange(0, 1, 0.1), xTrue, startingClockParams, satellitePositions, measurementNoiseSigma)
print("measurements with noise:")
print(measurements)

measurements with noise:
[[2.13956184e+07 5.13749953e+07 8.13543807e+07 1.11333767e+08
  1.41313137e+08 1.71292518e+08 2.01271909e+08 2.31251285e+08
  2.61230658e+08 2.91210045e+08]
 [2.18869073e+07 5.18662880e+07 8.18456730e+07 1.11825041e+08
  1.41804423e+08 1.71783810e+08 2.01763188e+08 2.31742577e+08
  2.61721950e+08 2.91701328e+08]
 [2.20906074e+07 5.20699780e+07 8.20493604e+07 1.12028732e+08
  1.42008118e+08 1.71987503e+08 2.01966876e+08 2.31946266e+08
  2.61925641e+08 2.91905023e+08]
 [2.26625667e+07 5.26419621e+07 8.26213316e+07 1.12600706e+08
  1.42580098e+08 1.72559466e+08 2.02538856e+08 2.32518223e+08
  2.62497608e+08 2.92476998e+08]
 [2.17022719e+07 5.16816492e+07 8.16610281e+07 1.11640408e+08
  1.41619781e+08 1.71599167e+08 2.01578549e+08 2.31557939e+08
  2.61537315e+08 2.91516683e+08]
 [2.34566098e+07 5.34359862e+07 8.34153652e+07 1.13394754e+08
  1.43374137e+08 1.73353517e+08 2.03332887e+08 2.33312271e+08
  2.63291655e+08 2.93271040e+08]
 [2.39379025e+07 5.39172849e+07 8

In [23]:
# These would be the Kalman filter matrices

# NOTE: this should probably be a class object

def estimatePseudorange(state, almanac):
    """
    Given a 5D state (x,y,z,time offset, freq) and an almanac/constellation,
    the function returns a corresponding pseudorange
    """
    # TODO: check if pseudorange has a frequency bias component?
    return np.linalg.norm(almanac - state[0:3], axis = 1) + state[3]
    
    
def _G(prevState, almanac):
    """
    Given a previous state and almanac, it returns a measurement connection matrix 
    (G in Kalman filtering literature). Basically, this is an estimate of what our
    measurements should be at a given state
    """
    nSatellites = almanac.shape[0]
    losVectors = almanac - prevState[0:3]
    norm = np.linalg.norm(losVectors, axis = 1)
    losNormalized = np.divide(-losVectors, norm[:,None])
    G = np.concatenate(-losNormalized, np.ones((nSatellites, 1)), axis=1)
    return G

def _kalmanGain(P, G, R):
    PdotGT = np.dot(P, G.T)
    A = np.linalg.inv(np.dot(G, PdotGT) + R)
    K = np.dot(PdotGT, A)
    return K

def _innovation(measurement, x, almanac):
    return measurement - estimatePseudorange(x, almanac)

def _stateEstimate(x, K, innovation):
    # estimate update
    xNew = x + np.dot(K, innovation)
    return newState

def _errorCovarUpdate():
    raise NotImplementedError
    
def _project():
    raise NotImplementedError
    

centroid = np.mean(satellitePositions, axis =0)
xInit = np.append(centroid, (0.0, const.c))
print("initial state: ", xInit)
print()

G = _G(xInit, satellitePositions)
innovation = _innovation(measurements[:,0], xInit, satellitePositions)
print("innovation: ", innovation)

initial state:  [ 1.91325077e+07 -5.77367543e+05 -5.95410963e+06  0.00000000e+00
  2.99792458e+08]

innovation:  [ 9372699.10533523 11502877.84330543  -699632.56600937  4567514.59634515
  2416545.37122554  7186005.86102337  3750471.49356441]
